# Analyze terrain-related controls on dDEMs

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xdem
import geoutils as gu
import seaborn as sns

In [ ]:
# Define input and output paths for convenience
data_path = '/Volumes/LaCie/raineyaberle/Research/PhD/SkySat-Stereo/study-sites/'
out_path = '/Users/raineyaberle/Research/PhD/SnowDEMs/skysat-snow/figures/'
plt.rcParams.update({'font.size':10, 'font.sans-serif': 'Arial'})

# Define function for plotting
def plot_ddem_vs_terrain(dem_fn, ddem_fn, refdem_fn, ss_mask_fn):
    print('Loading input files')
    dem = xdem.DEM(dem_fn)
    ddem = xdem.DEM(ddem_fn)
    ss_mask = gu.Raster(ss_mask_fn).reproject(ddem)
    ss_mask = (ss_mask==1)

    print('Calculating terrain parameters')
    refdem = xdem.DEM(refdem_fn).reproject(res=10)
    slope = refdem.slope().reproject(ddem)
    aspect = refdem.aspect().reproject(ddem)
    curv = refdem.curvature().reproject(ddem)
    refdem = refdem.reproject(ddem)

    print('Masking stable surfaces')
    dem = dem.reproject(ddem)
    dem.set_mask(~ss_mask)
    ddem.set_mask(~ss_mask)
    ddem_ss_med = np.nanmedian(ddem)
    ddem_ss_nmad = xdem.spatialstats.nmad(ddem)

    print('Prepping data for plotting')
    # Compile in dataframe
    df = pd.DataFrame({
        'elevation': refdem.data.ravel(),
        'slope': slope.data.ravel(),
        'aspect': aspect.data.ravel(),
        'max_curvature': curv.data.ravel(),
        'dDEM': ddem.data.ravel()
        })
    df.dropna(inplace=True)
    # Create nice bins for plotting
    nbins = 20
    elev_min = np.floor(df['elevation'].min() / 100) * 100
    elev_max = np.ceil(df['elevation'].max() / 100) * 100
    df['elevation_bin'] = pd.cut(df['elevation'], bins=np.linspace(elev_min, elev_max, nbins + 1))
    slope_min = np.floor(df['slope'].min() / 10) * 10
    slope_max = np.ceil(df['slope'].max() / 10) * 10
    df['slope_bin'] = pd.cut(df['slope'], bins=np.linspace(slope_min, slope_max, nbins + 1))
    df['aspect_bin'] = pd.cut(df['aspect'], bins=np.linspace(0, 360, nbins + 1))
    df['max_curvature_bin'] = pd.cut(df['max_curvature'], bins=np.linspace(-1, 1, nbins + 1))

    print('Plotting')
    fig, ax = plt.subplots(5, 2, figsize=(10,24))
    # map views
    ddem.plot(ax=ax[0,1], cmap='coolwarm_r', vmin=-5, vmax=5)
    ax[0,1].set_title('dDEM')
    for i, (raster, cmap, title) in enumerate([[dem, 'terrain', 'DEM'],
                                               [refdem, 'terrain', 'Elevation'],
                                               [slope, 'Oranges', 'Slope'],
                                               [aspect, 'twilight', 'Aspect'],
                                               [curv, 'RdGy_r', 'Max. curvature']]):
        if title=='Max. curvature':
            raster.plot(ax=ax[i,0], cmap=cmap, vmin=-1, vmax=1)
        else:
            raster.plot(ax=ax[i,0], cmap=cmap)
        ax[i,0].set_title(title)
        ax[i,0].set_xticks([])
        ax[i,0].set_yticks([])
    for i, column in enumerate(['elevation', 'slope', 'aspect', 'max_curvature']):
        # histogram
        ax2 = ax[i+1,1].twinx()
        bin_counts = df[column+'_bin'].value_counts(sort=False)
        ax2.bar(bin_counts.index.astype(str), bin_counts.values, color='k', alpha=0.5, width=1.0)
        # Set y-axis label and color for counts
        ax2.set_yticks([])
        ax2.set_ylim(0, np.nanmax(bin_counts.values)*3)
        # boxplot
        sns.boxplot(data=df, x=column+'_bin', y='dDEM', showfliers=False, ax=ax[i+1,1])
        ax[i+1,1].set_xlabel(column)
        ax[i+1,1].set_xticks(ax[i+1,1].get_xticks())
        ax[i+1,1].set_xticklabels(ax[i+1,1].get_xticklabels(), rotation=45)
        ax[i+1,1].axhline(y=0, xmin=ax[i+1,1].get_xlim()[0], xmax=ax[i+1,1].get_xlim()[1], 
                          color='k', linewidth=1)
        
    fig.suptitle(f'SS Median = {np.round(ddem_ss_med,3)} m, SS NMAD = {np.round(ddem_ss_nmad, 3)} m')
    fig.tight_layout()
    plt.show()

    return fig


In [ ]:
##### MCS 20240420 #####
site_name = "MCS"
date = "20240420"
refdem_fn = os.path.join(data_path, site_name, 'refdem', 'MCS_REFDEM_WGS84.tif')
dem_fn = os.path.join(data_path, site_name, date, f'{site_name}_{date}_DEM.tif')
ddem_fn = os.path.join(data_path, site_name, date, 'skysat_snow_nofilter', 'corr_coreg_diff', 'final_ddem.tif')
ss_mask_fn = os.path.join(data_path, site_name, date, 'skysat_snow_nofilter', 'land_cover_masks', 'stable_surfaces_mask.tif')

fig = plot_ddem_vs_terrain(dem_fn, ddem_fn, refdem_fn, ss_mask_fn)
fig_fn = os.path.join(out_path, f'ddem_stable_surfaces_vs_terrain_{site_name}_{date}.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)

In [ ]:
##### MCS 20241003 #####
site_name = "MCS"
date = "20241003"
refdem_fn = os.path.join(data_path, site_name, 'refdem', 'MCS_REFDEM_WGS84.tif')
dem_fn = os.path.join(data_path, site_name, date, f'{site_name}_{date}_DEM.tif')
ddem_fn = os.path.join(data_path, site_name, date, 'skysat_snow_nofilter', 'corr_coreg_diff', 'final_ddem.tif')
ss_mask_fn = os.path.join(data_path, site_name, date, 'skysat_snow_nofilter', 'land_cover_masks', 'stable_surfaces_mask.tif')

fig = plot_ddem_vs_terrain(dem_fn, ddem_fn, refdem_fn, ss_mask_fn)
fig_fn = os.path.join(out_path, f'ddem_stable_surfaces_vs_terrain_{site_name}_{date}.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)

In [ ]:
##### Banner 20240419-1 #####
site_name = "Banner"
date = "20240419-1"
refdem_fn = os.path.join(data_path, site_name, 'refdem', 'Banner_REFDEM_WGS84.tif')
dem_fn = os.path.join(data_path, site_name, date, f'{site_name}_{date}_DEM.tif')
ddem_fn = os.path.join(data_path, site_name, date, 'skysat_snow_nofilter', 'corr_coreg_diff', 'final_ddem.tif')
ss_mask_fn = os.path.join(data_path, site_name, date, 'skysat_snow_nofilter', 'land_cover_masks', 'stable_surfaces_mask.tif')

fig = plot_ddem_vs_terrain(dem_fn, ddem_fn, refdem_fn, ss_mask_fn)
fig_fn = os.path.join(out_path, f'ddem_stable_surfaces_vs_terrain_{site_name}_{date}.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)

In [ ]:
##### Banner 20240419-2 #####
site_name = "Banner"
date = "20240419-2"
refdem_fn = os.path.join(data_path, site_name, 'refdem', 'Banner_REFDEM_WGS84.tif')
dem_fn = os.path.join(data_path, site_name, date, f'{site_name}_{date}_DEM.tif')
ddem_fn = os.path.join(data_path, site_name, date, 'skysat_snow_nofilter', 'corr_coreg_diff', 'final_ddem.tif')
ss_mask_fn = os.path.join(data_path, site_name, date, 'skysat_snow_nofilter', 'land_cover_masks', 'stable_surfaces_mask.tif')

fig = plot_ddem_vs_terrain(dem_fn, ddem_fn, refdem_fn, ss_mask_fn)
fig_fn = os.path.join(out_path, f'ddem_stable_surfaces_vs_terrain_{site_name}_{date}.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)

In [ ]:
##### Jackson Creek 20240420 #####
site_name = "JacksonCreek"
date = "20240420"
refdem_fn = os.path.join(data_path, site_name, 'refdem', 'USGS_LPC_ID_FEMAHQ_2018_D18_merged_filtered.tif')
dem_fn = os.path.join(data_path, site_name, date, f'{site_name}_{date}_DEM.tif')
ddem_fn = os.path.join(data_path, site_name, date, 'skysat_snow_nofilter', 'corr_coreg_diff', 'final_ddem.tif')
ss_mask_fn = os.path.join(data_path, site_name, date, 'skysat_snow_nofilter', 'land_cover_masks', 'stable_surfaces_mask.tif')

fig = plot_ddem_vs_terrain(dem_fn, ddem_fn, refdem_fn, ss_mask_fn)
fig_fn = os.path.join(out_path, f'ddem_stable_surfaces_vs_terrain_{site_name}_{date}.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)